In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import requests
import json

In [2]:
person_df = pd.read_csv('Data/Person_FARS.csv', encoding='cp1252')

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (15,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
person_df.head(10)

pers_df = person_df.filter(['ST_CASE','MAKENAME', 'MAK_MOD', 'AGE', 'SEXNAME', 'DOANAME', 'PER_TYPNAME'], axis=1)
pers_df

,ST_CASE,MAKENAME,MAK_MOD,AGE,SEXNAME,DOANAME,PER_TYPNAME
0,10001,Toyota,Toyota Camry,34,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
1,10001,Toyota,Toyota Camry,53,Male,Died at Scene,Passenger of a Motor Vehicle In-Transport
2,10001,Freightliner,Freightliner Medium/Heavy - CBE,59,Male,Not Applicable,Driver of a Motor Vehicle In-Transport
3,10002,BMW,BMW 3-series,42,Female,Died at Scene,Driver of a Motor Vehicle In-Transport
4,10002,Chevrolet,Chevrolet TrailBlazer (2003 on; for 2002 model...,54,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
...,...,...,...,...,...,...,...
82215,560120,Hyundai,Hyundai Santa Fe,31,Male,Died at Scene,Driver of a Motor Vehicle In-Transport
82216,560120,Hyundai,Hyundai Santa Fe,22,Male,Not Applicable,Passenger of a Motor Vehicle In-Transport
82217,560121,Dodge,Dodge Dart (2013 on. See model 001 for 1960-19...,42,Male,Died at Scene,Driver of a Motor Vehicle In-Transport
82218,560121,Dodge,Dodge Dart (2013 on. See model 001 for 1960-19...,41,Female,Died at Scene,Passenger of a Motor Vehicle In-Transport


In [4]:
na_pers = pers_df[~pers_df.isin(["NOT APPLICABLE", "Not Reported", "Unknown", "Other", 999, 998]).any(axis=1)]
only_drivers = na_pers.loc[na_pers['PER_TYPNAME'] == "Driver of a Motor Vehicle In-Transport"]
only_drivers["PER_TYPNAME"].value_counts()

Driver of a Motor Vehicle In-Transport    49664
Name: PER_TYPNAME, dtype: int64

In [5]:
only_drivers.head()

,ST_CASE,MAKENAME,MAK_MOD,AGE,SEXNAME,DOANAME,PER_TYPNAME
0,10001,Toyota,Toyota Camry,34,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
2,10001,Freightliner,Freightliner Medium/Heavy - CBE,59,Male,Not Applicable,Driver of a Motor Vehicle In-Transport
3,10002,BMW,BMW 3-series,42,Female,Died at Scene,Driver of a Motor Vehicle In-Transport
4,10002,Chevrolet,Chevrolet TrailBlazer (2003 on; for 2002 model...,54,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
5,10003,Acura,Acura RL/RLX,22,Male,Not Applicable,Driver of a Motor Vehicle In-Transport


In [6]:
sc_pers_df = only_drivers.rename(columns={"ST_CASE": "Case_Number", "MAKENAME": "Vehicle_Model", "MAK_MOD": "Vehicle_Manufacturer",
                                             "SEXNAME": "Sex", "DOANAME": "DOA_Status", "PER_TYPNAME": "Passenger_Type"})

cleaned_pers_df = sc_pers_df.replace({'Not Applicable': 'No Fatality', 'Died at Scene': 'Fatal', 'nan': 'No Fatality', 'Died En Route': 'Fatal'})

cleaned_pers_df

,Case_Number,Vehicle_Model,Vehicle_Manufacturer,AGE,Sex,DOA_Status,Passenger_Type
0,10001,Toyota,Toyota Camry,34,Female,No Fatality,Driver of a Motor Vehicle In-Transport
2,10001,Freightliner,Freightliner Medium/Heavy - CBE,59,Male,No Fatality,Driver of a Motor Vehicle In-Transport
3,10002,BMW,BMW 3-series,42,Female,Fatal,Driver of a Motor Vehicle In-Transport
4,10002,Chevrolet,Chevrolet TrailBlazer (2003 on; for 2002 model...,54,Female,No Fatality,Driver of a Motor Vehicle In-Transport
5,10003,Acura,Acura RL/RLX,22,Male,No Fatality,Driver of a Motor Vehicle In-Transport
...,...,...,...,...,...,...,...
82213,560118,Peterbilt,Peterbilt Medium/Heavy - CBE,56,Male,No Fatality,Driver of a Motor Vehicle In-Transport
82214,560119,Harley-Davidson,Harley-Davidson 750cc or greater,52,Male,No Fatality,Driver of a Motor Vehicle In-Transport
82215,560120,Hyundai,Hyundai Santa Fe,31,Male,Fatal,Driver of a Motor Vehicle In-Transport
82217,560121,Dodge,Dodge Dart (2013 on. See model 001 for 1960-19...,42,Male,Fatal,Driver of a Motor Vehicle In-Transport


In [7]:
# Establish bins for ages

for row in cleaned_pers_df.iteritems():
    cleaned_pers_df['Age_Bin'] = pd.cut(cleaned_pers_df['AGE'],7, precision=0)
    cleaned_pers_df['Age_Label'] = pd.cut(cleaned_pers_df['AGE'],7, precision=0, labels=["1", "2", "3", "4", "5", "6", "7"])
    

In [13]:
cleaned_pers_df.head()

,Case_Number,Vehicle_Model,Vehicle_Manufacturer,AGE,Sex,DOA_Status,Passenger_Type,Age_Label
0,10001,Toyota,Toyota Camry,34,Female,No Fatality,Driver of a Motor Vehicle In-Transport,3
2,10001,Freightliner,Freightliner Medium/Heavy - CBE,59,Male,No Fatality,Driver of a Motor Vehicle In-Transport,4
3,10002,BMW,BMW 3-series,42,Female,Fatal,Driver of a Motor Vehicle In-Transport,3
4,10002,Chevrolet,Chevrolet TrailBlazer (2003 on; for 2002 model...,54,Female,No Fatality,Driver of a Motor Vehicle In-Transport,4
5,10003,Acura,Acura RL/RLX,22,Male,No Fatality,Driver of a Motor Vehicle In-Transport,2


In [11]:
cleaned_pers_df.drop(columns=["Age_Bin"], inplace=True)
cleaned_pers_df.dtypes

Case_Number                int64
Vehicle_Model             object
Vehicle_Manufacturer      object
AGE                        int64
Sex                       object
DOA_Status                object
Passenger_Type            object
Age_Label               category
dtype: object

In [12]:
# Writing Accidents final dataframe to a csv
cleaned_pers_df.to_csv(r"Output/person_final.csv", index=False, header=True)